In [2]:
import pandas as pd

In [86]:
root_table = 'E:/PUBLIC_DATA_PROJECT/SCRAPING_CODE/Optimizing_scraping/crawl_table'

In [87]:
df_after = pd.read_csv("test_kqkt.csv")
df_after = df_after.loc[df_after["stock_code"] == "HHG"]
df_after

,tax_code,stock_code,type,year,range,Trong đó: Chi phí lãi vay_23_0,Trong đó: Chi phí lãi vay_23_1,Trong đó: Chi phí lãi vay_23_LK0,Trong đó: Chi phí lãi vay_23_LK1,Doanh thu bán hàng và cung cấp dịch vụ_1_0,...,Lợi nhuận sau thuế_21_LK0,Lợi nhuận sau thuế_21_LK1,Lợi ích của cổ đông thiểu số_22_0,Lợi ích của cổ đông thiểu số_22_1,Lợi ích của cổ đông thiểu số_22_LK0,Lợi ích của cổ đông thiểu số_22_LK1,Lãi cơ bản trên cổ phiếu_23_0,Lãi cơ bản trên cổ phiếu_23_1,Lãi cơ bản trên cổ phiếu_23_LK0,Lãi cơ bản trên cổ phiếu_23_LK1
0,1000272301,HHG,Riêng,2017.0,Năm,"18,500,648,153","14,052,648,113",NaN,NaN,"346,101,283,293",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000272301,HHG,Riêng,2018.0,Năm,"18,348,027,793","18,500,648,153",NaN,NaN,"262,414,253,391",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000272301,HHG,Riêng,2019.0,Quý I,"4,632,195,319",0,"7,841,421,084",0,"55,324,734,583",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000272301,HHG,Riêng,2020.0,Quý I,"3,379,962,429","2,723,060,430","3,379,962,429","2,723,060,430","33,411,376,611",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000272301,HHG,Riêng,2021.0,Quý I,"2,576,813,312","2,996,189,687","4,853,557,554","6,376,152,116","25,630,749,144",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1000272301,HHG,Riêng,2021.0,Quý IV,"870,668,970","3,561,603,844","6,872,484,195","12,603,420,261","26,231,535,830",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1000272301,HHG,Riêng,2022.0,Bán niên,"3,632,472,071","4,853,542,513",NaN,NaN,"56,119,831,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1000272301,HHG,Riêng,2022.0,Quý I,"1,608,498,636","1,148,272,712","5,240,970,707","6,001,815,225","28,838,069,423",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1000272301,HHG,Riêng,2023.0,Quý IV,"1,550,677,783","1,518,410,890","4,715,239,035","6,759,381,597","11,943,715,507",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1000272301,HHG,Riêng,2024.0,Quý II,"377,666,511","2,030,445,206","1,052,629,841","2,830,884,697","11,416,948,119",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
df_before = pd.read_csv(root_table + "/HHG/success.csv")
df_before

,Unnamed: 0,STT,Tên báo cáo,Đơn vị,Trích yếu,Thời gian gửi,Mã doanh nghiệp,Tên công ty,Tiêu đề,Saving_path
0,0,1,Báo cáo tài chính quý 3/ 2024,CTCP Hoàng Hà,,18/10/2024 00:00:00,1000272301,CTCP Hoàng Hà,Báo cáo tài chính quý 3/ 2024,./crawl_table/HHG/Báo cáo tài chính quý 3_ 202...
1,1,2,Báo cáo tài chính quý 3/ 2024 của công ty mẹ,CTCP Hoàng Hà,,18/10/2024 00:00:00,1000272301,CTCP Hoàng Hà,Báo cáo tài chính quý 3/ 2024 của công ty mẹ,./crawl_table/HHG/Báo cáo tài chính quý 3_ 202...
2,2,3,Báo cáo tài chính bán niên 2024,CTCP Hoàng Hà,,16/08/2024 00:00:00,1000272301,CTCP Hoàng Hà,Báo cáo tài chính bán niên 2024,./crawl_table/HHG/Báo cáo tài chính bán niên 2...
3,3,4,Báo cáo tài chính quý 2/ 2024,CTCP Hoàng Hà,,24/07/2024 00:00:00,1000272301,CTCP Hoàng Hà,Báo cáo tài chính quý 2/ 2024,./crawl_table/HHG/Báo cáo tài chính quý 2_ 202...
4,4,5,Báo cáo tài chính quý 2/ 2024 của công ty mẹ,CTCP Hoàng Hà,,24/07/2024 00:00:00,1000272301,CTCP Hoàng Hà,Báo cáo tài chính quý 2/ 2024 của công ty mẹ,./crawl_table/HHG/Báo cáo tài chính quý 2_ 202...
...,...,...,...,...,...,...,...,...,...,...
69,69,70,Báo cáo tài chính Riêng quý III/ 2018,CTCP Hoàng Hà,Báo cáo tài chính quý III năm 2018,19/10/2018 17:51:23,1000272301,CTCP Hoàng Hà,Báo cáo tài chính Riêng quý III/ 2018,./crawl_table/HHG/Báo cáo tài chính quý III nă...
70,70,71,Báo cáo tài chính Riêng bán niên 2018,CTCP Hoàng Hà,Báo cáo tài chính giữa niên độ đã soát xét cho...,15/08/2018 17:49:13,1000272301,CTCP Hoàng Hà,Báo cáo tài chính Riêng bán niên 2018,./crawl_table/HHG/Báo cáo tài chính Riêng bán ...
71,71,72,Báo cáo tài chính Riêng quý II/ 2018,CTCP Hoàng Hà,Báo cáo tài chính Qúy II.2018,11/08/2018 10:05:39,1000272301,CTCP Hoàng Hà,Báo cáo tài chính Riêng quý II/ 2018,./crawl_table/HHG/Báo cáo tài chính Qúy II.201...
72,72,73,Báo cáo tài chính Riêng năm 2017,CTCP Hoàng Hà,Báo cáo tài chính năm 2017 đã được kiểm toán,30/03/2018 09:38:09,1000272301,CTCP Hoàng Hà,Báo cáo tài chính Riêng năm 2017,./crawl_table/HHG/Báo cáo tài chính năm 2017 đ...


In [89]:
def extract_quarter(text):
    """Hàm chuẩn hóa cách viết quý, bán niên, năm"""
    mapping = {
        "quý 1": "Quý I", "quý i/": "Quý I",
        "quý 2": "Quý II", "quý ii/": "Quý II",
        "quý 3": "Quý III", "quý iii/": "Quý III",
        "quý 4": "Quý IV", "quý iv/": "Quý IV",
        "bán niên": "Bán niên",
        "năm": "Năm"
    }
    text = text.lower().strip()
    for key, val in mapping.items():
        if key in text:
            return val
    return None


def extract_type(text):
    """Hàm chuẩn hóa cách viết quý, bán niên, năm"""
    mapping = {
        "riêng": "Riêng", "Riêng": "Riêng",
        "tổng hợp": "Tổng hợp", "Tổng hợp": "Tổng hợp",
        "Mẹ": "Riêng", "mẹ": "Riêng",
        "hợp nhất": "Tổng hợp", "Hợp nhất": "Tổng hợp"
    }
    text = text.lower().strip()
    for key, val in mapping.items():
        if key in text:
            return val
    return None

def map_source(df_before, df_after):
    # 1️⃣ Trích xuất year & range từ "Tên báo cáo" của df_before
    df_before["year"] = df_before["Tên báo cáo"].str.extract(r'(\d{4})').astype(str)
    df_after["year"] = df_after["year"].astype(str).str.extract(r'(\d{4})')

    df_before["range"] = df_before["Tên báo cáo"].apply(extract_quarter)

    # Trích xuất "type" từ "Tên báo cáo"
    df_before["type"] = df_before["Tên báo cáo"].apply(extract_type).fillna("Riêng")

    # 3️⃣ Tạo key chung để merge
    df_before["key"] = df_before["year"] + "_" + df_before["range"] + "_" + df_before["type"]
    df_after["key"] = df_after["year"] + "_" + df_after["range"] + "_" + df_after["type"]

    df_merged = df_before.merge(df_after[["key", "tax_code"]], on="key", how="left")

    print("Nb source report: ", len(df_merged))
    print("Nb check report: ", len(df_after))

    df_merged = df_merged.loc[df_merged["tax_code"] != "nan"]
    
    print("Nb mapped report: ", len(df_merged))

    return df_merged
